In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant P data

### Reading the files and extracting relevant information:

In [5]:
plant = "P"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  P ['INSTRUÇÕES', 'Clínquer', '175']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('P', '175', array(['CPIIF40'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIF40': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Composição"].rename(
            {"Sulfato de cálcio": "Sulfato de cálcio (composição)"}, axis=1
        ),
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Clínquer": "Clinker",
    "Fíler calcário": "Calcareous filler",
    "Cinza volante": "Fly ash",
    "Escória": "Slag",
    "Argila calcinada": "Calcined clay",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
    "Sulfato de cálcio (composição)": "Calcium sulfate (composition)",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Quartz,100.000000
Calcium sulfate,100.000000
Anhydrite,100.000000
Gamma C2S,100.000000
Beta C2S,100.000000
Alpha C2S,100.000000
Muscovite,100.000000
CS1,100.000000
Calcined clay,100.000000
Slag,100.000000


<IPython.core.display.Javascript object>

In [19]:
for var, val in (
    (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).items()
):
    s = f"""
    <tr>
        <td>{var}</td>
        <td>{round(val, 2)}</td>
    </tr>"""
    if val >= 40:
        # print(s)
        print(f'"{var}",')

"Quartz",
"Calcium sulfate",
"Anhydrite",
"Gamma C2S",
"Beta C2S",
"Alpha C2S",
"Muscovite",
"CS1",
"Calcined clay",
"Slag",
"Fly ash",
"Calcareous filler",
"Clinker",
"Specific Gravity",


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant P - Full Dataset:</th>
    <th></th>
  <tr>
        <td>Quartz</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcium sulfate</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Anhydrite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Gamma C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Beta C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Alpha C2S</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Muscovite</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>CS1</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcined clay</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Slag</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Fly ash</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Calcareous filler</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Clinker</td>
        <td>100.0</td>
    </tr>
    <tr>
        <td>Specific Gravity</td>
        <td>100.0</td>
    </tr>
</table>

In [20]:
FEATRUES_TO_DROP = [
    "Quartz",
    "Calcium sulfate",
    "Anhydrite",
    "Gamma C2S",
    "Beta C2S",
    "Alpha C2S",
    "Muscovite",
    "CS1",
    "Calcined clay",
    "Slag",
    "Fly ash",
    "Calcareous filler",
    "Clinker",
    "Specific Gravity",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [21]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for P
#200,88.888889
Dolimita,46.296296
Langbeinite,5.864198
Orthorhombic C3A,2.469136
#325,0.925926
Bassanite,0.308642
Aphthitalite,0.308642
Calcium sulfate (composition),0.000000
TiO2,0.000000
Fe2O3,0.000000


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>#200</td>
        <td>88.89>
    </tr>
    <tr>
        <td>Dolimita</td>
        <td>46.30</td>
    </tr>
</table>

In [22]:
FEATRUES_TO_DROP = ["Dolimita", "#200"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [23]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [24]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [25]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [26]:
df.shape

(324, 36)

<IPython.core.display.Javascript object>

In [27]:
# df.describe().T

<IPython.core.display.Javascript object>

In [28]:
# df_copy = df.copy()

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [29]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [30]:
df.shape

(294, 36)

<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Bassanite,25.510204
Langbeinite,18.707483
Portlandite,18.707483
Periclase,18.707483
Calcite,17.006803
Total C3S,17.006803
Total C2S,17.006803
C4AF,17.006803
C3A,17.006803
Cubic C3A,17.006803


<IPython.core.display.Javascript object>

In [32]:
for var, val in (
    (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).items()
):
    s = f"""
    <tr>
        <td>{var}</td>
        <td>{round(val, 2)}</td>
    </tr>"""
    if val >= 17:
        # print(s)
        print(f'"{var}",')

"Bassanite",
"Langbeinite",
"Portlandite",
"Periclase",
"Calcite",
"Total C3S",
"Total C2S",
"C4AF",
"C3A",
"Cubic C3A",
"Orthorhombic C3A",
"Free CaO",
"Arcanite",
"Aphthitalite",
"Calcium sulfate (composition)",


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Bassanite</td>
        <td>25.51</td>
    </tr>
    <tr>
        <td>Langbeinite</td>
        <td>18.71</td>
    </tr>
    <tr>
        <td>Portlandite</td>
        <td>18.71</td>
    </tr>
    <tr>
        <td>Periclase</td>
        <td>18.71</td>
    </tr>
    <tr>
        <td>Calcite</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Total C3S</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Total C2S</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>C4AF</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>C3A</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Cubic C3A</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Orthorhombic C3A</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Free CaO</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Arcanite</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Aphthitalite</td>
        <td>17.01</td>
    </tr>
    <tr>
        <td>Calcium sulfate (composition)</td>
        <td>17.01</td>
    </tr>
</table>


<p>UserWarning: Skipping features without any observed values:</p>
<p>['Calcium sulfate (composition)' 'Total C3S' 'Total C2S' 'C4AF' 'C3A'/<p>
 <p>'Cubic C3A' 'Orthorhombic C3A' 'Free CaO' 'Portlandite' 'Periclase'</p>
 <p>'Arcanite' 'Aphthitalite' 'Langbeinite' 'Bassanite' 'Calcite']. </p>
 <p>At least one non-missing value is needed for imputation with strategy='median'.</p>


In [33]:
FEATRUES_TO_DROP = [
    "Bassanite",
    "Langbeinite",
    "Portlandite",
    "Periclase",
    "Calcite",
    "Total C3S",
    "Total C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Arcanite",
    "Aphthitalite",
    "Calcium sulfate (composition)",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [34]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Loss on Ignition,0.340136
Insoluble Residue,0.340136
CS28,0.000000
CS7,0.000000
CS3,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#400,0.000000
#325,0.000000
Blaine,0.000000


<IPython.core.display.Javascript object>

In [35]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,294,2022-02-28 08:48:58.775510272,2021-01-07 00:00:00,2021-07-29 12:00:00,2022-02-27 12:00:00,2022-09-27 18:00:00,2023-05-02 00:00:00,NaN
CaO,294.0,59.932672,57.82,59.4325,60.08,60.4875,61.96,0.821701
MgO,294.0,5.298232,4.12,5.03,5.33,5.58,6.18,0.399083
Na2O,294.0,0.056483,0.02,0.04,0.05,0.07,0.16,0.028214
Al2O3,294.0,4.355249,3.88,4.26,4.36,4.43,5.03,0.150082
SiO2,294.0,17.850076,15.85,17.31,17.865,18.325,20.79,0.759
SO3,294.0,3.369922,2.61,3.18,3.35,3.55,4.18,0.261153
K2O,294.0,0.897766,0.74,0.8,0.9,0.97,1.16,0.10586
TiO2,294.0,0.214883,0.19,0.21,0.22,0.22,0.23,0.008169
Fe2O3,294.0,2.579959,2.32,2.5,2.58,2.63,3.09,0.115795


<IPython.core.display.Javascript object>

In [36]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,294,NaN,NaN,NaN,2022-02-28 08:48:58.775510272,2021-01-07 00:00:00,2021-07-29 12:00:00,2022-02-27 12:00:00,2022-09-27 18:00:00,2023-05-02 00:00:00
Cement_Type,294,1,CP II-F-40,294,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 0 to 323
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  294 non-null    datetime64[ns]
 1   CaO                   294 non-null    float64       
 2   MgO                   294 non-null    float64       
 3   Na2O                  294 non-null    float64       
 4   Al2O3                 294 non-null    float64       
 5   SiO2                  294 non-null    float64       
 6   SO3                   294 non-null    float64       
 7   K2O                   294 non-null    float64       
 8   TiO2                  294 non-null    float64       
 9   Fe2O3                 294 non-null    float64       
 10  Loss on Ignition      293 non-null    float64       
 11  Insoluble Residue     293 non-null    float64       
 12  Blaine                294 non-null    float64       
 13  #325                  294

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [38]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [39]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [40]:
df.to_csv("../../../data/interim/209/p.csv", index=False)

<IPython.core.display.Javascript object>